In [1]:
import os
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# torch imports
import torch
import torch.nn as nn

## CARLA dataset

In [2]:
from config import GlobalConfig
from data import CARLA_Data

root_dir = '/home/surya/Downloads/transfuser-2022/data/'
config = GlobalConfig(root_dir=root_dir, setting='all')
train_set = CARLA_Data(root=config.train_data, config=config)
val_set = CARLA_Data(root=config.val_data, config=config)
print(f"There are {len(train_set)} samples in training set")

100%|██████████| 11/11 [00:01<00:00,  8.98it/s]
Loading 16088 lidars from 11 folders
100%|██████████| 8/8 [00:00<00:00,  9.93it/s]
Loading 11543 lidars from 8 folders
There are 16088 samples in training set


Create pytorch style dataloaders

In [3]:
from torch.utils.data import DataLoader

g_cuda = torch.Generator(device='cpu')
g_cuda.manual_seed(torch.initial_seed())

# We need to seed the workers individually otherwise random processes 
# in the dataloader return the same values across workers!
def seed_worker(worker_id):
    # Torch initial seed is properly set across the different workers,
    # we need to pass it to numpy and random.
    worker_seed = (torch.initial_seed()) % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


dataloader_train = DataLoader(train_set, shuffle=True, batch_size=2, worker_init_fn=seed_worker, generator=g_cuda, num_workers=4)
dataloader_valid   = DataLoader(val_set, shuffle=False, batch_size=2, worker_init_fn=seed_worker, generator=g_cuda, num_workers=4)


In [4]:
sample_data = next(iter(dataloader_train))
print(f"sample data is of type {type(sample_data)} and has following keys")

for k,v in sample_data.items():
    print(k, list(v.shape))

sample data is of type <class 'dict'> and has following keys
rgb [2, 3, 160, 704]
bev [2, 160, 160]
depth [2, 160, 704]
semantic [2, 160, 704]
steer [2]
throttle [2]
brake [2]
speed [2]
theta [2]
x_command [2]
y_command [2]
light [2]
target_point [2, 2]
target_point_image [2, 1, 256, 256]
lidar [2, 2, 256, 256]
label [2, 20, 7]
ego_waypoint [2, 4, 2]


## Helper functions

In [5]:
def train_validate_model(model, num_epochs, model_name, optimizer, 
                         device, dataloader_train, dataloader_valid, 
                         lr_scheduler = None, output_path = '.'):

    # initialize placeholders for running values
    train_results = []    
    val_results = []    
    min_val_loss = np.Inf

    # move model to device
    model.to(device)
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_detailed_train_losses  = {key: 0.0 for key in config.detailed_losses}
        epoch_detailed_train_losses['weighted_loss'] = 0.0
        
        with tqdm(dataloader_train, unit="batch") as tepoch:
            for batch_idx, data in enumerate(tepoch):
                tepoch.set_description(f"Epoch {epoch}")

                # load data to gpu, according to type
                for k in ['rgb', 'depth', 'lidar', 'label', 'ego_waypoint', \
                          'target_point', 'target_point_image', 'speed']:
                    data[k] = data[k].to(device, torch.float32)
                for k in ['semantic', 'bev']:
                    data[k] = data[k].to(device, torch.long)
                
                # forward pass, store losses
                losses = model(data)
                loss = torch.tensor(0.0).to(device, dtype=torch.float32)
                for key, value in losses.items():
                    loss += detailed_weights[key] * value
                    epoch_detailed_train_losses[key] += float(detailed_weights[key] * value.item())
                epoch_detailed_train_losses['weighted_loss'] += float(loss.item())
                
                # backward pass
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()
                
                # log losses
                tepoch.set_postfix(loss=loss.item())

                if batch_idx == 2:
                    break
            
            # average losses across batches
            for k,v in epoch_detailed_train_losses.items():
                epoch_detailed_train_losses[k] = v / len(dataloader_train)
            
#         validation_loss, validation_metric = evaluate_model(
#                         model, dataloader_valid, criterion, metric_class, num_classes, device)

        
        train_results.append(epoch_detailed_train_losses)
    
    return train_results, val_results

## Model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [7]:
from model import LidarCenterNet
model = LidarCenterNet(config, device, config.backbone, image_architecture='regnety_032', 
                           lidar_architecture='regnety_032', use_velocity=False)
model.to(device);

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print ('Total trainable parameters: ', params)

Total trainable parameters:  168018327


In [8]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

## Training

In [9]:
N_EPOCHS = 1

detailed_weights = {key: config.detailed_losses_weights[idx] for idx, key in enumerate(config.detailed_losses)}

In [10]:
train_results, val_results = train_validate_model(model, num_epochs=N_EPOCHS, model_name='Transfuser_regnet032', 
                                                  optimizer=optimizer,device = device, dataloader_train=dataloader_train, 
                                                  dataloader_valid = dataloader_valid)

Epoch 0:   0%|          | 2/8044 [00:16<18:37:29,  8.34s/batch, loss=302] 


In [12]:
train_results

[{'loss_wp': 0.0006791537137603002,
  'loss_bev': 0.00040492759244113583,
  'loss_depth': 0.0017792778427715861,
  'loss_semantic': 0.0007342136303386186,
  'loss_center_heatmap': 0.06452217045143314,
  'loss_wh': 4.535051852775889e-05,
  'loss_offset': 2.235929462697481e-05,
  'loss_yaw_class': 0.0003740036683198767,
  'loss_yaw_res': 1.0020414951985308e-08,
  'loss_velocity': 0.0,
  'loss_brake': 0.0,
  'weighted_loss': 0.06856146669696066}]

In [16]:
import pandas as pd
train_results = pd.DataFrame(train_results)
train_results

,loss_wp,loss_bev,loss_depth,loss_semantic,loss_center_heatmap,loss_wh,loss_offset,loss_yaw_class,loss_yaw_res,loss_velocity,loss_brake,weighted_loss
0,0.000679,0.000405,0.001779,0.000734,0.064522,0.000045,0.000022,0.000374,1.002041e-08,0.0,0.0,0.068561
